In [1]:
import os
import sys
import pandas
import numpy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark

from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vector, Vectors
from pyspark.ml.feature import StandardScaler
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [2]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [3]:
file_name = "/Users/simondi/PROJECTS/target_infect_x_project/results/2-analysis/1-clustering/pca_transform-cells_sample_10_normalized_cut_100_K005"

In [4]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [5]:
data = spark.read.parquet(file_name)

In [6]:
data.take(1)

[Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='4', object_idx='144', prediction=0, features=DenseVector([-0.8044, 0.0121, 1.1159, 1.4749, -0.9369, -0.7485, -1.0209, -0.703, 0.0, 0.4476, 1.2809, 1.3916, 0.1489, 0.6694, -1.2335, -0.0825, 0.0106, -0.5078, 1.2455, 0.1357]), scaledFeatures=DenseVector([-0.8334, 0.1165, 1.0605, 1.3942, -0.9828, -0.7089, -1.0943, -0.8341, 0.0, 0.4385, 1.3451, 1.3012, 0.089, 0.7667, -1.357, -0.2354, -0.0203, -0.4495, 1.409, 0.085]), pcs=DenseVector([-0.2773, -2.0598]))]

In [35]:
data.select(["gene", "prediction"]).dropDuplicates().take(5)

[Row(gene='chka', prediction=3),
 Row(gene='chka', prediction=0),
 Row(gene='ckm', prediction=2),
 Row(gene='mknk2', prediction=3),
 Row(gene='none', prediction=2)]

In [61]:
filt_dat = data.groupby(["gene", "prediction"]).count()
a = filt_dat.select(["gene", "prediction", "count"]).dropDuplicates().toPandas()

In [64]:
a.to_csv("/Users/simondi/Desktop/pups.ts")

In [10]:
data.filter("gene='chka'").collect()


[Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='4', object_idx='144', prediction=0, features=DenseVector([-0.8044, 0.0121, 1.1159, 1.4749, -0.9369, -0.7485, -1.0209, -0.703, 0.0, 0.4476, 1.2809, 1.3916, 0.1489, 0.6694, -1.2335, -0.0825, 0.0106, -0.5078, 1.2455, 0.1357]), scaledFeatures=DenseVector([-0.8334, 0.1165, 1.0605, 1.3942, -0.9828, -0.7089, -1.0943, -0.8341, 0.0, 0.4385, 1.3451, 1.3012, 0.089, 0.7667, -1.357, -0.2354, -0.0203, -0.4495, 1.409, 0.085]), pcs=DenseVector([-0.2773, -2.0598])),
 Row(study='infectx_published', pathogen='listeria', library='a', design='u', replicate='1', plate='kb2-02-1x', well='a01', gene='chka', sirna='s3008', well_type='sirna', image_idx='5', object_idx='168', prediction=0, features=DenseVector([-0.9474, -0.0399, 0.1262, 0.6198, -1.0081, -0.7959, -0.9485, -0.6324, 0.0, 0.4263, -0.2104, -0.3939, 0.3941, -0.3786, 1.0053